In [1]:
import cx_Oracle
import os
import pandas as pd
#!pip insall cx_Oracle

In [2]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [3]:
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

## change_cell_content
1. 사용자의 셀 내용 변경은 DB에 반영이 되지 않는다.
    - 모든 업무시작시 db의 sht내용은 저장된다
2. 셀변경은 하나의 셀만 변경 가능
    - 두 개이상의 셀선택시 사용 불가!
3. change_cell 모드에서도 idx, 입고일, 출고일, pod_date는 변경할 수 없다.


### 과정
1. 특정 셀을 클릭한 후 select_cell 버튼 클릭
2. 해당셀의 주소와 내용이 특정 cell에 표시된다.
3. change_cell 버튼을 누르면 input_box가 나와 값을 입력 받는다.
4. 이때 해당 cell의 index값을 찾아 db table 내용 update를 진행한다.
5. **DB 내용 UPDATE 과정**이 완료된 후에만 실제 xl내용에 반영하도록한다.
    - 아닐 시 error 리턴

In [4]:
def select_cell():
    sel_cells = wb_cy.selection
    sel_sht = wb_cy.selection.sheet
    # 선택한 셀의 row번호가 10미만이면 종료 ==> table값은 row가 10부터 시작이기 때문
    if wb_cy.selection.row < 10 :
        wb_cy.app.alert("선택한 셀은 바꿀 수 없습니다. 매서드를 종료합니다.","Change Cell WARNING")
        return None
    address_cell = sel_sht.range("E3")
    from_cell = sel_sht.range("E4")
    # 선택한셀의 value가 list type 이면 두 개이상의 셀을 선택 했다는 것 ==> 종료
    if type(sel_cells.value) is list :
        wb_cy.app.alert("하나의 셀만 선택 후 진행해주세요. 두 개 이상은 불가합니다.","Change Cell WARNING")
        return None
    
    address_cell.value = str(sel_cells.address)
    from_cell.value = sel_cells.value

In [12]:
def change_cell():
    sel_sht = wb_cy.selection.sheet
    address_cell = sel_sht.range("E3")
    from_cell = sel_sht.range("E4")
    change_cell_list = sel_sht.range("E3:E4")
    tb_name = sel_sht.range("D5").value
    idx_col_name = sel_sht.range("A9").value
    
    # 셀주소가 빈값이면 중지한다.
    if address_cell.value == None:
        wb_cy.app.alert("바꿀 셀이 없습니다 매서드를 종료합니다","Change Cell WARNING")
        return None
    
    xl_from_cell = sel_sht.range(address_cell.value)
    to_cell = wb_cy.app.api.InputBox("바꿀 내용을 입력 해주세요", "Change Cell Input", Type=2)
    # to_cell == False면 입력을 취소 했다는 뜻이므로 바꿀 뜻이 없는 것으로 간주하고 주소와 바뀔 값들의 form을 지운다.
    if to_cell == False :
        wb_cy.app.alert("취소를 선택하셨습니다. 셀 변경을 취소합니다.","Change Cell WARNING")
        change_cell_list.clear_contents()
        return None
    
    # DB UPDATE 진행
    row_num = sel_sht.range(address_cell.value).row
    col_mum = sel_sht.range(address_cell.value).column
    idx_num = sel_sht.range(row_num,1).options(numbers=int).value
    col_name = sel_sht.range(9,col_mum).options(numbers=int).value
    query = f"UPDATE {tb_name} SET {col_name} = '{to_cell}' WHERE {idx_col_name} = {idx_num}"
    cursor.execute(query)
    cursor.execute("commit")
    
    # DB UPDATE 완료 후 xl_cell 내용 변경
    xl_from_cell.value = to_cell
    
    # 모든게 완료 되면 change_cell_list 내용 모두삭제
    change_cell_list.clear_contents()
    # 변경 성공 메시지
    wb_cy.app.alert("셀 내용 변경이 완료되었습니다.","Change Cell Done")
    

In [10]:
select_cell()

In [11]:
change_cell()